In [56]:
from eth_account import Account
import eth_keys
from eth_keys.constants import SECPK1_N 
from eth_hash.auto import keccak 
from eth_keys.backends.native.ecdsa import decode_public_key, encode_raw_public_key, private_key_to_public_key
from eth_keys.backends.native.jacobian import fast_multiply, fast_add 
from  eth_utils.encoding import big_endian_to_int
# from eth_keys import private_key_to_public_key
import os



In [57]:
def generate_triple():
    priv = eth_keys.keys.PrivateKey(os.urandom(32))
    priv_hex = priv._raw_key.hex()

    pub = eth_keys.keys.PublicKey.from_private(priv)
    account = pub.to_checksum_address()
    return (priv, pub, account)


alice = generate_triple()

print(f"Alice's private key: {alice[0]}")
print(f"Alice's public key: {alice[1]}")
print(f"Alice's account's address: {alice[2]}")

Alice's private key: 0xae106d587189411bc9ad8899b88b5351beeea26978abb1d9e16d82ef2bcbc848
Alice's public key: 0xbc8f687b18107f57294fb7f6b925cfc1da7b32003fbd44d569c73060f45fa292dde1eb78f80ca0615aadea098bd4b583a11c6a0c12be025e74e74a7dade989ee
Alice's account's address: 0x6cBF738541d59F0E1CeC4b5B3c6DC500aF32eC6d


In [58]:
def   mult_priv_key_pub_key(private_key, public_key):
    # return private_key * public_key
    priv_as_number  = big_endian_to_int(private_key._raw_key)
    pubkey_as_tuple = decode_public_key(public_key._raw_key)
    new_point = fast_multiply(pubkey_as_tuple, priv_as_number)
    
    encoded_new_point = encode_raw_public_key(new_point)
    new_pubkey = eth_keys.keys.PublicKey(encoded_new_point)
    return new_pubkey

In [59]:
alice = generate_triple()
bob = generate_triple()

alice_priv = alice[0]
alice_pub = alice[1]

bob_priv = bob[0]
bob_pub = bob[1]

bob_priv_alice_pub = mult_priv_key_pub_key(bob_priv, alice_pub)
alice_priv_bob_pub = mult_priv_key_pub_key(alice_priv, bob_pub)

print(f"Bob's private key * Alice's public key: {bob_priv_alice_pub}")
print(f"Alice's private key * Bob's public key: {alice_priv_bob_pub}")

print(f"Bob's private key * Alice's public key == Alice's private key * Bob's public key: {bob_priv_alice_pub == alice_priv_bob_pub}")

Bob's private key * Alice's public key: 0x92630b331ecfbba82de5fe09249909b2503b1664603608875b4511e2676bfb1a45b8d3cd75a812abc114a612be9a2850405afca7c17094c5ab4042c0e143137e
Alice's private key * Bob's public key: 0x92630b331ecfbba82de5fe09249909b2503b1664603608875b4511e2676bfb1a45b8d3cd75a812abc114a612be9a2850405afca7c17094c5ab4042c0e143137e
Bob's private key * Alice's public key == Alice's private key * Bob's public key: True


In [60]:
def secret_to_hash(secret):
    return big_endian_to_int(keccak(secret))

print(f"the number version of the hash of Bob's private key * Alice's public key: {secret_to_hash(bob_priv_alice_pub._raw_key)}")

the number version of the hash of Bob's private key * Alice's public key: 92392652484788306995279264890571604930265393532005835043227996712844974116360


In [61]:
def new_pubkey_from_secret(secret,pubkey):
    if isinstance(pubkey, eth_keys.keys.PublicKey):
        pubkey = decode_public_key(pubkey._raw_key)
    else:
        raise TypeError("pubkey must be a PublicKey object")
    numeric_secret = secret_to_hash(secret) % SECPK1_N
    numeric_secret_bytes = numeric_secret.to_bytes(32, byteorder='big')
    secret_pubkey = private_key_to_public_key(numeric_secret_bytes)
    secret_pubkey_tuple = decode_public_key(secret_pubkey)
    secret_point = fast_add(pubkey, secret_pubkey_tuple)

    encoded_secret_point = encode_raw_public_key(secret_point)
    new_pubkey = eth_keys.keys.PublicKey(encoded_secret_point)

    return new_pubkey

def new_address_from_secret(secret, pubkey):
    if not isinstance(pubkey, eth_keys.keys.PublicKey):
        raise TypeError("pubkey must be a PublicKey object")

    new_pubkey = new_pubkey_from_secret(secret, pubkey)
    return new_pubkey.to_checksum_address()

def new_private_key_from_secret(secret, private_key):
    if not isinstance(private_key, eth_keys.keys.PrivateKey):
        raise TypeError("private_key must be a PrivateKey object")
    numeric_secret = secret_to_hash(secret)
    private_key_as_number = big_endian_to_int(private_key._raw_key)
    new_private_key_as_number = (private_key_as_number + numeric_secret) % SECPK1_N

    new_private_key_as_bytes = new_private_key_as_number.to_bytes(32, byteorder='big')

    new_private_key = eth_keys.keys.PrivateKey(new_private_key_as_bytes)
    return new_private_key

In [62]:
secret = bob_priv_alice_pub._raw_key
new_pubkey = new_pubkey_from_secret(secret, bob_pub)
print(f"the new public key: {new_pubkey}")

# private key with secret
new_private_key = new_private_key_from_secret(secret, bob_priv)
# private key to public key
new_pubkey2 = eth_keys.keys.PublicKey(private_key_to_public_key(new_private_key._raw_key))

print(f"the new public key: {new_pubkey2}")
# are thw two pubkeys the same?
print(f"are the two pubkeys the same? {new_pubkey == new_pubkey2}")


the new public key: 0xedd3258e89baa2f7c5072b43f772de77d07c745515d0696ead0993089ad4e75c3b243fed3919454c92df4f032607e3cfb5a2f7bdaaf7858a02c399190528470d
the new public key: 0xedd3258e89baa2f7c5072b43f772de77d07c745515d0696ead0993089ad4e75c3b243fed3919454c92df4f032607e3cfb5a2f7bdaaf7858a02c399190528470d
are the two pubkeys the same? True
